In [29]:
## Text classification with TensorFlow Hub: Movie reviews
# This is an example of binary—or two-class—classification
# TensorFlow Hub and Keras
# use the IMDB dataset that contains the text of 50,000 movie reviews from the Internet Movie Database. 
# These are split into 25,000 reviews for training and 25,000 reviews for testing. 
# The training and testing sets are balanced, meaning they contain an equal number of positive and negative reviews.
# TensorFlow Hub is a library for the publication, discovery, and consumption of reusable parts of machine learning models.

import numpy as np 
import tensorflow as tf
!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [30]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.2.0-dev20200501
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [31]:
# Download the IMDB dataset

# Split the training set into 60% and 40%, 15,000 examples for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [32]:
# Explore the data
# The label is an integer value of either 0 or 1, where 0 is a negative review, and 1 is a positive review.

train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))  # Let's print first 10 examples.
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [33]:
train_labels_batch  # Let's also print the first 10 labels. 

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [36]:
# Build the model
# The neural network is created by stacking layers—this requires three main architectural decisions:
# 1) How to represent the text?
# 2) How many layers to use in the model?
# 3) How many hidden units to use for each layer?
# In this example, the input data consists of sentences. 
# The labels to predict are either 0 or 1.
# Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences, 
# and try it out on a couple of input examples. 
# Note that no matter the length of the input text, the output shape of the embeddings is: (num_examples, embedding_dimension).

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [37]:
# Let's now build the full model:

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_5 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Let's compile the model.
# Loss function and optimizer
# A model needs a loss function and an optimizer for training.

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [41]:
# Train the model
# Train the model for 20 epochs in mini-batches of 512 samples. 
# This is 20 iterations over all samples in the x_train and y_train tensors.

history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Instructions for updating:
Use `tf.data.Dataset.cardinality()


Instructions for updating:
Use `tf.data.Dataset.cardinality()


Epoch 1/20
30/30 [==============================] - 2s 68ms/step - loss: 0.7353 - accuracy: 0.4994 - val_loss: 0.6793 - val_accuracy: 0.5490
Epoch 2/20
30/30 [==============================] - 2s 62ms/step - loss: 0.6600 - accuracy: 0.5691 - val_loss: 0.6395 - val_accuracy: 0.5915
Epoch 3/20
30/30 [==============================] - 2s 66ms/step - loss: 0.6214 - accuracy: 0.6137 - val_loss: 0.6087 - val_accuracy: 0.6416
Epoch 4/20
30/30 [==============================] - 2s 64ms/step - loss: 0.5871 - accuracy: 0.6537 - val_loss: 0.5740 - val_accuracy: 0.6788
Epoch 5/20
30/30 [==============================] - 2s 66ms/step - loss: 0.5443 - accuracy: 0.7049 - val_loss: 0.5332 - val_accuracy: 0.7110
Epoch 6/20
30/30 [==============================] - 2s 65ms/step - loss: 0.5011 - accuracy: 0.7390 - val_loss: 0.4955 - val_accuracy: 0.7475
Epoch 7/20
30/30 [==============================] - 2s 63ms/step - loss: 0.4608 - accuracy: 0.7732 - val_loss: 0.4621 - val_accuracy: 0.7784
Epoch 8/20
30

In [43]:
# Evaluate the model
# Loss (a number which represents our error, lower values are better), and accuracy.

results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3166 - accuracy: 0.8642
loss: 0.317
accuracy: 0.864
